타이타닉 생존자 예측

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#데이터 로딩
titanic_df = pd.read_csv('../00.data/titanic/train.csv')
titanic_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
# 데이터 확인
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# 데이터 확인결과 :Age,Cabin,Embarked 의 null 값(결손값) 존재.  Sex,Cabin,Embarked문자화되어있다.  

In [6]:
# 데이터 전처리

In [7]:
    ## 결손값 채우기 -> filna 로 채우면 int로 된다
titanic_df['Age'].fillna(titanic_df['Age'].mean(),inplace=True)
titanic_df['Cabin'].fillna('N',inplace=True)
titanic_df['Embarked'].fillna('N',inplace=True)

In [8]:
        ## 다 채워졌는지  확인 --> False 이면 공백이 없다
titanic_df.isnull().any().any()

False

In [9]:
         ##공백의 갯수는 몇개인가?
 titanic_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [10]:
    ## 문자를 수치화

In [12]:
            ### 캐빈의 첫글자만 추출
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1] 
print(titanic_df['Cabin'].head(3))

0    N
1    C
2    N
Name: Cabin, dtype: object


In [13]:
            ### Sex,Cabin,Embarked문자화되어 있는것을 라벨화해서 수치로 변경

In [14]:
from sklearn.preprocessing import LabelEncoder

items = ['Cabin', 'Sex', 'Embarked']
for item in items:
    encoder = LabelEncoder() 
    titanic_df[item] = encoder.fit_transform(titanic_df[item])

titanic_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,7,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,2,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,7,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,2,3
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,7,3


In [24]:
#Survived 속성을 y로 나머지 피쳐를 X로 만듦 titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop(['Survived','Name','Ticket'], axis=1)


In [ ]:
#features = ['PassengerId', 'Pclass', 'Sex','Age', 'SibSp',  'Fare','Cabin' , 'Embarked_encode']
#X_titanic = titanic_df[numeric]
#  X_titanic_df = titanic_df.drop('Survived', axis=1)

In [25]:
#학습/테스트 데이터 세트 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

In [20]:
#적용할 알고리즘 선정 : • 결정 트리 – DecisionTreeClassifier  • 랜덤 포레스트 – RandomForestClassifier  • 로지스틱 회귀 – LogisticRegression 

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [27]:
#학습, 예측 및 평가 

In [28]:
# DecisionTreeClassifier 학습/예측/평가
dt_clf = DecisionTreeClassifier(random_state=11)
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print(f'DecisionTreeClassifier 정확도: {accuracy_score(y_test, dt_pred):.4f}') 

# RandomForestClassifier 학습/예측/평가
rf_clf = RandomForestClassifier(random_state=11)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print(f'RandomForestClassifier 정확도:{accuracy_score(y_test, rf_pred):.4f}')

# LogisticRegression 학습/예측/평가
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print(f'LogisticRegression 정확도: {accuracy_score(y_test, lr_pred):.4f}')

# SupportVectorClassifier 학습/예측/평가
sv_clf = SVC(random_state=11)
sv_clf.fit(X_train, y_train)
sv_pred = sv_clf.predict(X_test)
print(f'SupportVectorClassifier 정확도: {accuracy_score(y_test, sv_pred):.4f}')

DecisionTreeClassifier 정확도: 0.7318
RandomForestClassifier 정확도:0.8492
LogisticRegression 정확도: 0.8212
SupportVectorClassifier 정확도: 0.7151


In [29]:
#교차 검증 :결정 트리 모델을 cross_val_score 함수를 이용하여 교차 검증

In [30]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores):
    print(f"교차 검증 {iter_count} 정확도: {accuracy:.4f}")

print(f"평균 정확도: {np.mean(scores):.4f}")

교차 검증 0 정확도: 0.6034
교차 검증 1 정확도: 0.7809
교차 검증 2 정확도: 0.8090
교차 검증 3 정확도: 0.7921
교차 검증 4 정확도: 0.8202
평균 정확도: 0.7611


In [31]:
#GridSearchCV를 통해 최적 파라미터 도출

In [32]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth':[2,3,5,10],  #여기의 숫자값은 어떻게 구하는가?
    'min_samples_leaf':[1,5,8],
    'min_samples_split':[2,3,5] 
}
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_)
print(f'GridSearchCV 최고 정확도: {grid_dclf.best_score_:.4f}')
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
pred = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'테스트 세트에서의 DecisionTreeClassifier 정확도: {accuracy:.4f}')

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.7978
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.8603
